## 1. Get Followers

In [ ]:
# fixes some issues when running twint in Notebook
import nest_asyncio
nest_asyncio.apply()

In [ ]:
import twint

In [ ]:
# Configure
c = twint.Config()

c.Username = 'hgmaassen'
c.Proxy_host = 'tor'
c.Store_csv = True
c.Output = 'followers.csv'

In [ ]:
twint.run.Followers(c)

## 2. Get Retweets of Followers

Fetching the follower could have been done with the official API as well. Originally, I wanted to everything with twint but Twitter blocked the API for the requests.

In [ ]:
from pathlib import Path
import twitter
import json

In [ ]:
followers = Path('followers.csv').read_text().split()[1:]

In [ ]:
api = twitter.Api(
    '', '', '', '', sleep_on_rate_limit=True)

In [ ]:
def get_tweets(api=None, screen_name=None):
    timeline = api.GetUserTimeline(screen_name=screen_name, count=200)
    if len(timeline) == 0:
        return []
    print(len(timeline))
    earliest_tweet = min(timeline, key=lambda x: x.id).id
    print("getting tweets before:", earliest_tweet)

    while True:
        tweets = api.GetUserTimeline(
            screen_name=screen_name, max_id=earliest_tweet, count=200
        )
        new_earliest = min(tweets, key=lambda x: x.id).id

        if not tweets or new_earliest == earliest_tweet:
            break
        else:
            earliest_tweet = new_earliest
            print("getting tweets before:", earliest_tweet)
            timeline += tweets

    return timeline

In [ ]:
for f in followers:
    if Path(f'/mnt/data/datasets/twitter/tweets-maassen/{f}.json').is_file():
        continue
    print(f)
    try:
        timeline = get_tweets(api=api, screen_name=f)
        with open(f'/mnt/data/datasets/twitter/tweets-maassen/{f}.json', 'w+') as f:
            for tweet in timeline:
                f.write(json.dumps(tweet._json))
                f.write('\n')
    except Exception as e:
        print(e)